# VisoMaster Internal Guide & Diagnostics

**Welcome!** This notebook helps you verify your VisoMaster instance setup on Vast.ai, check logs, and perform basic troubleshooting.

**Current Configuration:** (This needs to be determined dynamically or set based on the build)
- **Build Type:** [Default/Alt/Small-Default/Small-Alt] 
- **Access:** [VNC/Portal]
- **Models/Assets:** [Built-in/Downloaded at Runtime]

---

## 1. Environment & Path Checks

In [ ]:
import os
import sys

print(f"Python Executable: {sys.executable}")
print(f"Current Working Dir: {os.getcwd()}")
print("\nEnvironment Variables (PATH):")
print(os.environ.get('PATH'))
print("\nConda Environment:")
!conda info --envs

## 2. Check Key Directories

In [ ]:
print("--- Checking /app structure ---")
!ls -l /app

print("\n--- Checking /app/VisoMaster --- ")
!ls -l /app/VisoMaster

print("\n--- Checking /app/dependencies --- ")
# This might be empty for s-def/s-alt builds until the entrypoint runs
!ls -l /app/dependencies

print("\n--- Checking /app/models --- ")
# This might be empty for s-def/s-alt builds until the entrypoint runs
!ls -l /app/models

print("\n--- Checking /workspace --- ")
# This is mounted from the Vast.ai host
!ls -l /workspace

## 3. Check GPU Availability (PyTorch Example)

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available! Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version (PyTorch): {torch.version.cuda}")
    print(f"cuDNN Version (PyTorch): {torch.backends.cudnn.version()}")
else:
    print("CUDA is NOT available. Check Vast.ai instance type and NVIDIA drivers.")

print("\n--- nvidia-smi Output ---")
!nvidia-smi

## 4. Check Running Services (Supervisor)

In [ ]:
# Requires supervisorctl to be configured correctly (unix socket)
!supervisorctl status

## 5. View Logs

Log locations depend on the configuration:
- **VNC Builds (`default`, `s-def`):** Primarily in `/var/log/supervisor/`
- **Portal Builds (`alt`, `s-alt`):** App/VNC/Caddy logs often in `/var/log/portal/`, supervisor core logs in `/var/log/supervisor/`

In [ ]:
print("--- Supervisord Log (Last 20 lines) ---")
!tail -n 20 /var/log/supervisor/supervisord.log

print("\n--- VisoMaster App Log (Last 20 lines) ---")
# Adjust path for portal builds if needed
!tail -n 20 /var/log/supervisor/visomaster_app.log || tail -n 20 /var/log/portal/visomaster_app.log

print("\n--- VisoMaster App Error Log (Last 20 lines) ---")
# Adjust path for portal builds if needed
!tail -n 20 /var/log/supervisor/visomaster_app_err.log || tail -n 20 /var/log/portal/visomaster_app_err.log

print("\n--- X11VNC Log (Last 20 lines) ---")
# Adjust path for portal builds if needed
!tail -n 20 /var/log/supervisor/x11vnc.log || tail -n 20 /var/log/portal/x11vnc.log

print("\n--- Caddy Log (Portal Builds Only - Last 20 lines) ---")
!tail -n 20 /var/log/portal/caddy.log || echo 'Caddy log not found (expected for VNC builds)'

## 6. Manual Actions (Use with Caution)

In [ ]:
# Example: Restart the VisoMaster app via supervisorctl
# !supervisorctl restart visomaster_app

# Example: Manually run the model download script (for s-def/s-alt)
# print("Running model download script manually...")
# !conda run -n visomaster python /app/VisoMaster/download_models.py --output_dir /app/models

# Example: Manually run the ffmpeg/ffplay download part (for s-def/s-alt)
# print("Running ffmpeg/ffplay download manually...")
# !wget -q -O /app/dependencies/ffmpeg.exe https://github.com/visomaster/visomaster-assets/releases/download/v0.1.7_dp/ffmpeg.exe
# !wget -q -O /app/dependencies/ffplay.exe https://github.com/visomaster/visomaster-assets/releases/download/v0.1.7_dp/ffplay.exe
# print("Done.")